## Extraction des tables d'irradiation via PVGIS, by departement in France.

We will create irradiance tables for each department in France with the help of PVGIS TMY service.
The geographical center of each department is stored in centres_geo.csv file.

We will export one csv file for each department in the irrad_tmy_base folder. 
Each file contains the irradiance components : global, direct and diffuse irradiance.

- sergiob

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
df = pd.read_csv("centres_geo.csv", ";")

# add leading zero to deps, drop non numeric codes like 2a or 2b
df = df[df["code"].apply(lambda x: x.isnumeric())]
df["code"] = pd.to_numeric(df['code'])
df['code'] = df['code'].apply(lambda x: '{0:0>2}'.format(x))

# tranform sexagesimal coords to decimal
def conversion(old):
    direction = {'N':1, 'S':-1, 'E': 1, 'O':-1}
    new = old.replace(u'°',' ').replace('\'',' ').replace('"',' ').replace('′',' ').replace('″',' ')
    new = new.split()
    new_dir = new.pop()
    new.extend([0,0,0])
    return (int(new[0])+int(new[1])/60.0+int(new[2])/3600.0) * direction[new_dir]

df["lat"] = df["gps sixagesimal"].str.split(', ',  expand=True)[0]
df["lon"] = df["gps sixagesimal"].str.split(', ',  expand=True)[1]
df["lat"] = df["lat"].apply(lambda x : conversion(x))
df["lon"] = df["lon"].apply(lambda x : conversion(x))

df.drop(["gps sixagesimal"], axis=1,  inplace=True)
df.drop(["label"], axis=1,  inplace=True)

In [3]:
df.head()

,code,lat,lon
0,01,46.099444,5.348889
1,02,49.559444,3.558333
2,03,46.393611,3.188333
3,04,44.106111,6.243889
4,05,44.663611,6.263056


In [4]:
import requests
from requests.exceptions import HTTPError
import time

def get_TMY_file(lat,lon):
        """Uses PVGIS webservice to create a Typical Meteorological Year (TMY)
         file using the PVSystem coordinates.
        more about TMY files
        https://ec.europa.eu/jrc/en/PVGIS/tools/tmy
        Return
        ------
        A dataframe instance consisting of 1 year (or several years) of hourly
        data,  with the following columns:
            "time_pvgis" : UTC for normal CSV, local timezone time
            "GHI" : Global horizontal irradiance G(h) in [W/m2].
            "DNI" : Direct (beam) irradiance Gb(n) in [W/m2].
            "DHI" : Diffuse horizontal irradiance Gd(h) in [W/m2].
        """

        url = "https://re.jrc.ec.europa.eu/api/tmy"

        params = {
            "lat": lat,
            "lon": lon,
            "startyear": 2006,
            "endyear": 2015,
            "outputformat": "json",  # csv, json, epw
        }

        start = time.time()

        try:
            r = requests.get(url, params=params)

            # If respons successful, no Exception will be raised
            r.raise_for_status()

        except HTTPError as http_err:
            print(f"HTTP error occurred: {http_err}")

        except Exception as err:
            print(f"Other error occurred: {err}")

        else:
#             print("get_TMY_file: done in {:.2f} seconds.".format(time.time() - start))

            tmy_json = r.json()
            df_r = pd.DataFrame.from_dict(data=tmy_json["outputs"]["tmy_hourly"])
            
            df_tmy = df_r[[ "G(h)", "Gb(n)", "Gd(h)"]]
            df_tmy.columns = ["GHI", "DNI", "DHI"]
            df_tmy.set_index(df_r["time(UTC)"], inplace=True)

            return df_tmy

In [5]:
%%time
for i, row in tqdm(df.iterrows(), total=df.shape[0]):
    code_dep = row["code"]
    lat, lon = row[["lat", "lon"]]    
    df = get_TMY_file(lat,lon)
    df.to_csv("./irrad_tmy_base/{}_irrad_tmy.csv".format(code_dep))

100%|██████████| 94/94 [04:22<00:00,  2.79s/it]Wall time: 4min 22s

